In [1]:
import optuna
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class TextValueDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row['generated_text']
        label = row['suddenness'] - 1  # Convert value to 0-4 classes
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load dataset
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

# Create DataLoader
def create_dataloader(df, tokenizer, max_length, batch_size):
    dataset = TextValueDataset(df, tokenizer, max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Define the model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

train_df = load_dataset('data/train.csv')
val_df = load_dataset('data/val.csv')
test_df = load_dataset('data/test.csv')

# Define model initialization function
def model_init():
    return RobertaForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Compute metrics function for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Base training arguments (will be modified by Optuna)
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"  # Used for early stopping
)

# Define the Optuna objective function
def objective(trial):
    # Define hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    max_length = trial.suggest_int("max_length", 64, 256, step=32)
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    
    # Update the training arguments with trial parameters
    training_args.learning_rate = learning_rate
    training_args.weight_decay = weight_decay
    training_args.per_device_train_batch_size = batch_size
    training_args.per_device_eval_batch_size = batch_size
    training_args.num_train_epochs = num_train_epochs
    
    # Create a dataloader dynamically for the different max_length
    train_loader = create_dataloader(train_df, tokenizer, max_length, batch_size)
    val_loader = create_dataloader(val_df, tokenizer, max_length, batch_size)
    test_loader = create_dataloader(test_df, tokenizer, max_length, batch_size)
    
    # Initialize Trainer with dynamic parameters
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_loader.dataset,
        eval_dataset=val_loader.dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    # Train the model
    trainer.train()
    
    # Evaluate the model and return validation loss for Optuna to minimize
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]

# Run the hyperparameter search with Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # Adjust n_trials based on resources

# Output the best parameters found by Optuna
print("Best hyperparameters: ", study.best_params)

/home/adebnath/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/adebnath/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[I 2024-10-29 21:40:19,175] A new study created in memory with name: no-name-00e6da03-2840-4888-aa65-8461e92c70ef
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.521100,1.419441,0.422222,0.286713,0.422222,0.330044
2,1.432800,1.467599,0.398148,0.344975,0.398148,0.362912
3,1.335300,1.517758,0.396296,0.379370,0.396296,0.371765
4,1.098000,1.638979,0.361111,0.403331,0.361111,0.361424
5,0.923700,1.870518,0.407407,0.388677,0.407407,0.396653
6,0.827000,2.130147,0.375926,0.450329,0.375926,0.397570
7,0.563200,2.486439,0.362963,0.406867,0.362963,0.376723
8,0.455100,2.851539,0.374074,0.435418,0.374074,0.393155
9,0.371000,3.180795,0.362963,0.404944,0.362963,0.378286
10,0.235600,3.461296,0.351852,0.396882,0.351852,0.368647


/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-10-29 21:57:48,673] Trial 0 finished with value: 1.4194414615631104 and parameters: {'learning_rate': 3.630902234210563e-05, 'weight_decay': 0.28787429340022275, 'batch_size': 8, 'max_length': 192, 'num_train_epochs': 10}. Best is trial 0 with value: 1.4194414615631104.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequence

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.516100,1.396965,0.409259,0.297981,0.409259,0.344197
2,1.370900,1.426372,0.396296,0.381567,0.396296,0.385737
3,1.257000,1.517614,0.387037,0.393731,0.387037,0.384472
4,0.969300,1.759681,0.351852,0.393475,0.351852,0.360038
5,0.825700,1.869668,0.366667,0.380856,0.366667,0.371915
6,0.716100,2.208246,0.362963,0.396681,0.362963,0.374735
7,0.482400,2.391719,0.355556,0.386506,0.355556,0.366715
8,0.415300,2.506539,0.361111,0.397065,0.361111,0.374382


...

/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-10-29 23:17:51,612] Trial 8 finished with value: 1.3819905519485474 and parameters: {'learning_rate': 2.0469844308717504e-05, 'weight_decay': 0.2130710341754846, 'batch_size': 8, 'max_length': 160, 'num_train_epochs': 6}. Best is trial 5 with value: 1.3779667615890503.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceC

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.389233,0.429630,0.303179,0.429630,0.349826
2,1.442700,1.408970,0.405556,0.383790,0.405556,0.387138
3,1.442700,1.468088,0.411111,0.399942,0.411111,0.404829
4,1.203500,1.671319,0.394444,0.422070,0.394444,0.395307
5,0.901400,1.940397,0.375926,0.383098,0.375926,0.378981
6,0.901400,2.166032,0.390741,0.428950,0.390741,0.403618
7,0.629100,2.457632,0.372222,0.421634,0.372222,0.387625
8,0.409400,2.528838,0.383333,0.425460,0.383333,0.397928


/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/home/adebnath/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-10-29 23:26:25,105] Trial 9 finished with value: 1.3892327547073364 and parameters: {'learning_rate': 3.2330403920463426e-05, 'weight_decay': 0.29669206872972104, 'batch_size': 16, 'max_length': 96, 'num_train_epochs': 8}. Best is trial 5 with value: 1.3779667615890503.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequence

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 10.57 GiB of which 16.00 MiB is free. Process 667779 has 2.89 GiB memory in use. Including non-PyTorch memory, this process has 7.66 GiB memory in use. Of the allocated memory 7.44 GiB is allocated by PyTorch, and 31.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)